<a href="https://colab.research.google.com/github/dakshitas/BlackScholes/blob/main/BSM_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from alpha_vantage.timeseries import TimeSeries
from scipy.stats import norm
from datetime import datetime, date
import time

In [7]:
ALPHA_VANTAGE_API_KEY = "P2TQ5YPY8ZUDAMT7"
ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format="pandas")

TODAY = datetime(2025, 2, 20)
risk_free_rate = 0.0495  # 10-Year Treasury Yield

def get_volatility(ticker):
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize="full")
        data = data.sort_index()
        data = data.iloc[-252:]
        returns = data["4. close"].pct_change().dropna()
        return returns.std() * np.sqrt(252)  # Annualized volatility
    except Exception as e:
        return None

# Function to fetch latest stock price
def get_stock_price(ticker):
    try:
        data, meta_data = ts.get_quote_endpoint(symbol=ticker)
        return float(data["05. price"].iloc[0])  # Fix: Use .iloc[0]
    except Exception as e:
        return None

# Black-Scholes Model
def black_scholes_model(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return np.round(price, 3)

ticker_select = widgets.Combobox(
    options=["AAPL", "MSFT", "JNJ", "GOOGL", "TSLA"],
    description="Ticker:",
    placeholder="Enter ticker"
)

maturity_picker = widgets.DatePicker(
    description="Maturity:"
)

strike_input = widgets.FloatText(
    description="Strike Price:",
    value=250
)

option_type_radio = widgets.RadioButtons(
    options=["call", "put"],
    description="Option Type:"
)

calculate_btn = widgets.Button(
    description="Calculate",
    button_style="primary"
)

def on_calculate_click(b):
    try:
        ticker = ticker_select.value
        if not ticker:
            display(HTML("<div style='color: red;'>Please enter a valid ticker!</div>"))
            return

        sigma = get_volatility(ticker)
        S = get_stock_price(ticker)

        if sigma is None or S is None:
            display(HTML(f"<div style='color: red;'>Failed to retrieve data for {ticker}. Try again later.</div>"))
            return

        K = strike_input.value
        maturity_date = maturity_picker.value

        if maturity_date is None:
            display(HTML("<div style='color: red;'>Please select a valid expiration date.</div>"))
            return

        # Fix: Convert datetime.date to datetime.datetime
        maturity_date = datetime.combine(maturity_date, datetime.min.time())

        if maturity_date <= TODAY:
            display(HTML("<div style='color: red;'>Maturity date must be in the future.</div>"))
            return

        T = (maturity_date - TODAY).days / 252  # Convert days to years
        r = risk_free_rate
        option_type = option_type_radio.value

        option_price = black_scholes_model(S, K, T, r, sigma, option_type)

        result_html = f"""
        <div style='color: green; font-weight: bold;'>
            Black-Scholes {option_type} option price for {ticker}: ${option_price}
        </div>
        """
        display(HTML(result_html))

    except Exception as e:
        display(HTML(f"<div style='color: red;'>Error: {e}</div>"))

calculate_btn.on_click(on_calculate_click)

display(ticker_select, maturity_picker, strike_input, option_type_radio, calculate_btn)

Combobox(value='', description='Ticker:', options=('AAPL', 'MSFT', 'JNJ', 'GOOGL', 'TSLA'), placeholder='Enter…

DatePicker(value=None, description='Maturity:')

FloatText(value=250.0, description='Strike Price:')

RadioButtons(description='Option Type:', options=('call', 'put'), value='call')

Button(button_style='primary', description='Calculate', style=ButtonStyle())